In [1]:
from PIL import Image
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import math
import random
import argparse
import logging
import json
import cv2
import datetime

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict

import tensorflow as tf
import keras
import ast
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Add, Input, Concatenate
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras import backend as K

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%mkdir /content/sample_data/output
%mkdir /content/sample_data/output/process_images
%mkdir /content/sample_data/output/csv
%cd /content/drive/My Drive/xview/model/


!python process_data.py --input_dir "/content/drive/My Drive/xview/test-data/data-split"  --output_dir "/content/sample_data/output/process_images" --output_dir_csv "/content/sample_data/output/csv"

/content/drive/My Drive/xview/model
INFO:root:Started Processing for Data
100% 1866/1866 [22:36<00:00,  1.38it/s]
INFO:root:Finished Processing Data


In [4]:
NUM_WORKERS = 4 
NUM_CLASSES = 4
BATCH_SIZE = 64
NUM_EPOCHS = 100 
LEARNING_RATE = 0.0001
RANDOM_SEED = 123
# LOG_STEP = 150
# LOG_DIR = '/path/to/logs' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

damage_intensity_encoding = dict()
damage_intensity_encoding[3] = '3'
damage_intensity_encoding[2] = '2' 
damage_intensity_encoding[1] = '1' 
damage_intensity_encoding[0] = '0' 

In [5]:
def validation_generator(test_csv, test_dir):
    df = pd.read_csv(test_csv)
    df = df.replace({"labels" : damage_intensity_encoding })

    gen = keras.preprocessing.image.ImageDataGenerator(
                             rescale=1/255.)


    return gen.flow_from_dataframe(dataframe=df,
                                   directory=test_dir,
                                   x_col='uuid',
                                   y_col='labels',
                                   batch_size=BATCH_SIZE,
                                   shuffle=False,
                                   seed=RANDOM_SEED,
                                   class_mode="categorical",
                                   target_size=(128, 128))

In [ ]:
weights = 'imagenet'
inputs = Input(shape=(128, 128, 3))

base_model = VGG16(include_top=False, weights=weights, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

x = Flatten()(x)

base_resnet = base_model(inputs)
base_resnet = Flatten()(base_resnet)

concated_layers = Concatenate()([x, base_resnet])

concated_layers = Dense(2024, activation='relu')(concated_layers)
concated_layers = Dense(524, activation='relu')(concated_layers)
concated_layers = Dense(124, activation='relu')(concated_layers)
output = Dense(4, activation='relu')(concated_layers)

model = Model(inputs=inputs, outputs=output)
model.load_weights("/content/drive/My Drive/xview/baseline_model/-saved-model-38-0.70.hdf5")

validation_gen = validation_generator("/content/sample_data/output/csv/train.csv", "/content/sample_data/output/process_images")
predictions = model.predict(validation_gen)

y_trues = validation_gen.classes
y_pred = np.argmax(predictions, axis=-1)



Found 54862 validated image filenames belonging to 4 classes.


In [ ]:
print("Evaluation of vgg16 model")
f1_weighted = f1_score(y_trues, y_pred, average='weighted')
print("f1 score is: "+str(f1_weighted))
precision = precision_score(y_trues, y_pred, average='weighted')
print("precision :"+str(precision))
recall =recall_score(y_trues, y_pred, average='weighted') 
print("recall :"+str(recall))
confusionMatrix = confusion_matrix(y_trues, y_pred)
print("\nconfusion Matrix :")
print(confusionMatrix)

Evaluation of vgg16 model
f1 score is: 0.718060185556541
precision :0.8267015464621181
recall :0.6783566038423682

confusion Matrix :
[[28611  5482  4113  4233]
 [  457  2810  1138   393]
 [  259   387  2777   427]
 [  178   220   359  3018]]


In [6]:
weights = 'imagenet'
inputs = Input(shape=(128, 128, 3))

base_model = VGG19(include_top=False, weights=weights, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

x = Flatten()(x)

base_resnet = base_model(inputs)
base_resnet = Flatten()(base_resnet)

concated_layers = Concatenate()([x, base_resnet])

concated_layers = Dense(2024, activation='relu')(concated_layers)
concated_layers = Dense(524, activation='relu')(concated_layers)
concated_layers = Dense(124, activation='relu')(concated_layers)
output = Dense(4, activation='relu')(concated_layers)

model = Model(inputs=inputs, outputs=output)
model.load_weights("/content/drive/My Drive/xview/baseline_model_vgg19/-saved-model-31-0.71.hdf5")

validation_gen = validation_generator("/content/sample_data/output/csv/train.csv", "/content/sample_data/output/process_images")
predictions = model.predict(validation_gen)

testTrues = validation_gen.classes
testPred = np.argmax(predictions, axis=-1)

80142336/80134624 [==============================] - 0s 0us/step
Found 54862 validated image filenames belonging to 4 classes.


In [12]:
print("Evaluation of Vgg19 model")
f1_weighted = f1_score(testTrues, testPred, average='weighted')
print("f1 score is: "+str(f1_weighted))
precision = precision_score(testTrues, testPred, average='weighted', labels=np.unique(test_pred))
print("precision :"+str(precision))
recall =recall_score(testTrues, testPred, average='weighted') 
print("recall :"+str(recall))
# accuracy = accuracy_score(test_trues, test_pred)
# # print("accuracy : "+str(accuracy))
confusionMatrix = confusion_matrix(testTrues, testPred)
print("\nconfusion Matrix :")
print(confusionMatrix)

Evaluation of Vgg19 model
f1 score is: 0.7420575721666249
precision :0.7603662853941788
recall :0.7484050891327331

confusion Matrix :
[[35258     0  3286  3895]
 [ 2042     0  1693  1063]
 [  667     0  2703   480]
 [  356     0   321  3098]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
weights = 'imagenet'
inputs = Input(shape=(128, 128, 3))

base_model = ResNet50(include_top=False, weights=weights, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

x = Flatten()(x)

base_resnet = base_model(inputs)
base_resnet = Flatten()(base_resnet)

concated_layers = Concatenate()([x, base_resnet])

concated_layers = Dense(2024, activation='relu')(concated_layers)
concated_layers = Dense(524, activation='relu')(concated_layers)
concated_layers = Dense(124, activation='relu')(concated_layers)
output = Dense(4, activation='relu')(concated_layers)
#output = concated_layers
model = Model(inputs=inputs, outputs=output)

#model.load_weights("/content/drive/My Drive/xview/classification.hdf5")
model.load_weights("/content/drive/My Drive/xview/classification_model/-saved-model-29-0.71.hdf5")
validation_gen = validation_generator("/content/sample_data/output/csv/train.csv", "/content/sample_data/output/process_images")
predictions = model.predict(validation_gen)

test_trues = validation_gen.classes
test_pred = np.argmax(predictions, axis=-1)


94773248/94765736 [==============================] - 0s 0us/step
Found 54862 validated image filenames belonging to 4 classes.


In [13]:
print("Evaluation of ResNet50 architecture")
f1_weighted = f1_score(testTrues, testPred, average='weighted', labels=np.unique(testPred))
print("f1 score is: "+str(f1_weighted))

precision = precision_score(testTrues, testPred, average='weighted', labels=np.unique(testPred))
print("precision : "+str(precision))
recall =recall_score(testTrues, testPred, average='weighted') 
print("recall : "+str(recall))
accuracy = accuracy_score(testTrues, testPred)
print("accuracy : "+str(accuracy))

print()
confusionMatrix = confusion_matrix(testTrues, testPred)
print(confusionMatrix)

Evaluation of ResNet50 architecture
f1 score is: 0.813174387268404
precision : 0.8332377586548305
recall : 0.7484050891327331
accuracy : 0.7484050891327331

[[35258     0  3286  3895]
 [ 2042     0  1693  1063]
 [  667     0  2703   480]
 [  356     0   321  3098]]
